# Learning Rate Testing Final


The best model was BeitLargePatch16 with validation accuracy 0.9856114983558655.
- We test with the BeitLargePatch16 model

Here we will experiment with the results of [learning-rate-test.ipynb](./learning-rate-test.ipynb)

In [1]:
import mlflow

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

import keras_tuner as kt
import tensorflow_addons as tfa

from tensorflow import keras
from keras.applications import imagenet_utils

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
DATASET_SIZE = 9367
IMAGE_SIZE = 224
BATCH_SIZE = 8
WORKERS = 4
EPOCHS = 10

BASE_PATH='../data'

classes = [
    'cup', 
    'fork', 
    'glass', 
    'knife', 
    'plate', 
    'spoon'
]

First, we will load the training dataframe and split it into train and validation

In [3]:
df_train_full = pd.read_csv('data/train.csv', dtype={'Id': str})
df_train_full['filename'] = 'data/images/' + df_train_full['Id'] + '.jpg'
df_train_full.head()

,Id,label,filename
0,0560,glass,data/images/0560.jpg
1,4675,cup,data/images/4675.jpg
2,0875,glass,data/images/0875.jpg
3,4436,spoon,data/images/4436.jpg
4,8265,plate,data/images/8265.jpg


In [4]:
val_cutoff = int(len(df_train_full) * 0.8)
df_train = df_train_full[:val_cutoff]
df_val = df_train_full[val_cutoff:]

## Learning Rate Search

Now let's create image generators

In [5]:
# These models don't have the imagenet preprocessing built in so I have to apply this
def preprocess_input(x, data_format=None):
    return imagenet_utils.preprocess_input(
        x, data_format=data_format, mode="tf"
    )

In [6]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    dtype="float16"
)

train_generator = train_datagen.flow_from_dataframe(
    df_train,
    x_col='filename',
    y_col='label',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    dtype="float16"
)

val_generator = val_datagen.flow_from_dataframe(
    df_val,
    x_col='filename',
    y_col='label',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
)

Found 4447 validated image filenames belonging to 6 classes.
Found 1112 validated image filenames belonging to 6 classes.


In [7]:
classes = np.array(list(train_generator.class_indices.keys()))
classes

array(['cup', 'fork', 'glass', 'knife', 'plate', 'spoon'], dtype='<U5')

In [8]:
earlystopping = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_accuracy',
    min_delta = 1e-4,
    patience = 3,
    mode = 'max',
    restore_best_weights = True,
    verbose = 1
)

callbacks = [earlystopping]

## Final Search with the BEiT model:

In [9]:
from keras_cv_attention_models import beit

In [14]:
def get_model():
    base_model = beit.BeitLargePatch16( #BeitBasePatch16(
        pretrained='imagenet21k-ft1k',  #weights='imagenet',
        input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
        num_classes=len(classes)
    )
    base_model.trainable = False
    # Removing the old classification head
    base_model = tf.keras.Model(inputs=base_model.layers[1].input, outputs= base_model.layers[-3].output)

    inputs = keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    # Adding our own classification head
    model = tf.keras.Sequential([
        inputs,
        base_model,
        tf.keras.layers.Flatten(),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(12, 'gelu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(6, 'softmax')
    ],
    name = 'efficientnet2_xl')

    return model

In [15]:
def build_model(hp):

    model = get_model()

    # Define the optimizer learning rate as a hyperparameter.
    learning_rate = hp.Choice("lr", values = [0.003, 0.001, 0.0003])
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

In [16]:
EXP_NAME = 'learning_rate_search_final'

tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=3,
    executions_per_trial=1,
    overwrite=False,
    directory='kt_' + EXP_NAME,
    tuner_id='kt_' + EXP_NAME,
    project_name='kt_' + EXP_NAME,
)

c:\Users\andre\anaconda3\envs\keras-attention-models\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


>>>> Load pretrained from: C:\Users\andre\.keras\models\beit_large_patch16_224_imagenet21k-ft1k.h5


In [17]:
tuner.search(
    x = train_generator,
    validation_data=val_generator,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    workers=WORKERS,
    callbacks=callbacks
)

Trial 3 Complete [00h 14m 27s]
val_accuracy: 0.9874100685119629

Best val_accuracy So Far: 0.9874100685119629
Total elapsed time: 00h 37m 39s
INFO:tensorflow:Oracle triggered exit


In [18]:
print(tuner.results_summary())

Results summary
Results in kt_learning_rate_search_final\kt_learning_rate_search_final
Showing 10 best trials
Trial summary
Hyperparameters:
lr: 0.0003
Score: 0.9874100685119629
Trial summary
Hyperparameters:
lr: 0.001
Score: 0.9847122430801392
Trial summary
Hyperparameters:
lr: 0.003
Score: 0.9847122430801392
None


The best result was a learning rate of **0.0003**